In [1]:
from bs4 import BeautifulSoup
from multiprocessing.pool import ThreadPool
import numpy as np
import pandas as pd
import make_data
import datetime
import requests
import glob
import sys
import os
import json
import time
import re

In [2]:
names = [405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 456, 419, 420, 421, 422, 838, 839, 840, 841, 842, 843, 884, 423, 424, 425, 426, 427, 1497, 428, 429, 430, 997, 1028, 1029, 1030, 1031, 1046, 1144, 1121, 1127, 1261, 1262, 1334, 1430]

In [3]:
len(names)

48

In [4]:
sm = 0
for i in names:
    f = open('data/categories/{}_urls.txt'.format(i))
    lines = f.readlines()
    sm += len(lines)
    f.close()
print(sm)

7238


In [6]:
tree = np.load('data/tree.npy')

In [ ]:
def dfs(tree, parents, childrens):
    mas = []
    if tree['children'] != None:
        for i in tree['children']:
            parents[i['a_attr']['title'].split('.')[0]] = tree['a_attr']['title'].split('.')[0]
            mas.extend([i['a_attr']['title'].split('.')[0]])
            dfs(i, parents, childrens)
    childrens[tree['a_attr']['title'].split('.')[0]] = mas

In [ ]:
parents = {'0':'0'}
childrens = {}
for i in tree:
    parents[i['a_attr']['title'].split('.')[0]] = '0'
    dfs(i, parents, childrens)

In [ ]:
def getData(tree, names, allUrls, parent_numb, st):
    dataUrls = []
    dataText = []
    dataCategories = []
    
    number = int(tree['a_attr']['title'].split('.')[0])
    if number in names:
        f1 = open('data/categories/{}_urls.txt'.format(number))
        f2 = open('data/categories/{}_text.txt'.format(number))
        categUrls = f1.readlines()
        categText = f2.readlines()
        f1.close()
        f2.close()
        
        if len(categUrls) <= 1:
#             print(number, len(categUrls))
            ...
        else:
            categUrls = [i.split()[0].replace('\n', '') for i in categUrls]

            for i in range(len(categUrls)):
                if categUrls[i] not in allUrls:
                    dataUrls.extend([categUrls[i]])
                    dataText.append([categText[i].replace('\n', '')])
                    dataCategories.extend([number])

                    allUrls.add(categUrls[i])
            
#             print(len(categUrls), '\t', len(categUrls) - len(dataUrls), '\t', \
#                   round((len(categUrls) - len(dataUrls)) / len(categUrls), 2), '\t', \
#                   st + tree['a_attr']['title'])

#     flag = True
    st += '   '
    if len(tree['children']) != 0:
        for i in tree['children']:
            a, b, c = getData(i, names, allUrls, number, st)
            dataUrls.extend(a)
            dataText.extend(b)
            dataCategories.extend(c)
    
    else:
        if number in names and (len(categUrls) - len(dataUrls)) / len(categUrls) > 0.8:
            dataCategories = (np.ones(len(dataCategories)) * parent_numb).tolist()
#             flag = False
    
#     if number in names and flag:
#         countUrl = dataCategories.count(number)
#         print(len(categUrls), '\t', len(categUrls) - countUrl, '\t', \
#               round((len(categUrls) - countUrl) / len(categUrls), 2), '\t', \
#               st + tree['a_attr']['title'])
    return dataUrls, dataText, dataCategories

In [ ]:
files = glob.glob('data/filterData/categories/*')
files = [int(i[27:].split('.')[0]) for i in files]

In [ ]:
allUrls = set()
dataUrls = []
dataText = []
dataCategories = []
st = '   '
parent_numb = 1
for i in tree:
    a, b, c = getData(i, names, allUrls, parent_numb, st)
    dataUrls.extend(a)
    dataText.extend(b)
    dataCategories.extend(c)

In [ ]:
dataUrls = np.array(dataUrls)
dataText = np.array(dataText,  dtype='O')
dataCategories = np.array(dataCategories)

In [ ]:
len(set(dataCategories))

In [ ]:
len(dataUrls)

In [ ]:
dataAns = [(dataUrls[i], dataCategories[i]) for i in range(dataUrls.shape[0])]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
trainText, testText, trainAns, testAns = \
    train_test_split(dataText, dataAns, test_size=0.2)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestCentroid, KNeighborsClassifier, NearestNeighbors
from sklearn.cross_validation import cross_val_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.linear_model import RidgeClassifier, LinearRegression, LogisticRegression

In [ ]:
trainCategory = [float(i[1]) for i in trainAns]
testCategory = [float(i[1]) for i in testAns]

In [ ]:
words = {}
for i in trainText:
    for j in i[0].split():
        if j in words:
            words[j] += 1
        else:
            words[j] = 1

In [ ]:
mas = [[x,y] for x,y in list(words.items())]
mas = np.array(mas, dtype='O')
wordsSort = mas[mas[:,1].argsort()]
oneWord = wordsSort[wordsSort[:,1] < 5][:,0]

stopWord = {i:0 for i in oneWord}

In [ ]:
for idx, i in enumerate(trainText):
    for j in i[0].split():
        if j in stopWord:
            stopWord[j] = idx

In [ ]:
_time = time.time()
for i in stopWord:
    trainText[stopWord[i], 0] = trainText[stopWord[i], 0].replace(i, '')
print(time.time() - _time)

In [ ]:
trainTexts = [i[0] for i in trainText]
testTexts = [i[0] for i in testText]

In [ ]:
tfidfTransformer = TfidfVectorizer()
trainFeatures  = tfidfTransformer.fit_transform(trainTexts)
testFeatures = tfidfTransformer.transform(testTexts)

In [ ]:
trainFeatures

# CLF

In [65]:
_time = time.time()
clf = RidgeClassifier()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  19.864394903182983
0.700298953662
0.7675635276532138


In [66]:
_time = time.time()
clf = NearestCentroid()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  0.10212588310241699
0.609118086697
0.7361733931240658


In [67]:
_time = time.time()
clf = KNeighborsClassifier()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  0.009233951568603516
0.287742899851
0.37144992526158443


In [68]:
_time = time.time()
clf = KNeighborsClassifier(algorithm='brute', metric='cosine')
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  0.008516788482666016
0.634529147982
0.7227204783258595


In [69]:
_time = time.time()
clf = SVC()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  76.14178729057312
0.130792227205
0.15994020926756353


In [70]:
_time = time.time()
clf = SVC(gamma=0.5)
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  95.05464029312134
0.654708520179
0.7451420029895366


In [61]:
def dfs(tree, st):
    nums = [tree['id']]
    text = [tree['text']]
    print(st + tree['a_attr']['title'])
    st += '    '
    if tree['children'] != None:
        for i in tree['children']:
            t = dfs(i, st)
            nums.extend(t[0])
            text.extend(t[1])
    return nums, text

In [62]:
tree = np.load('data/tree.npy')
nums = []
text = []
for i in tree:
    st = ''
    t = dfs(i, st)
    nums.extend(t[0])
    text.extend(t[1])

d = np.vstack((nums, text))
categories = dict(d.T)

91. Компьютеры
    111. Оргтехника (принтеры, сканеры, факсы)
    106. Аксессуары
    104. Информационные носители и внешние устройства чтения
    101. Периферийные устройства
        327. Клавиатуры, мыши, комплекты
        331. Компьютерная акустика(наушники, колонки) и микрофоны
        332. Мониторы
        335. Прочие периферийные устройства
    94. Комплектующие
        346. Жесткие диски и сетевые накопители
        348. Видеокарты
        352. Процессоры
        401. Материнские платы
        798. Прочие комплектующие
    92. ПК
        245. Моноблоки и стационарные компьютеры
        325. Ноутбуки, нетбуки
    97. Сетевое оборудование
        1198. Настройка сетевого оборудования
    1124. Ремонт компьютеров
        1126. Тест веб-камеры
        1205. Советы по настройке и использованию ПК
            1206. Настройка Windows
    1183. Программное обеспечение
        1204. Драйверы
1186. Эмиграция
    1187. Учеба за границей
    1188. Работа за границей
1190. Сценическое, студи